# **Stratified Model Training for Multiclass SQL Injection Detection**
We sampled according to the original class distribution and split the dataset into training, validation, and test sets using a 70%-10%-20% ratio. shown in the paper. For example, 6.85% of the training set was drawn from “Union-based” SQL injection, 6.74% from “Stack queries-based” SQL injection, and so on. The validation and test sets followed the same distribution.

In [1]:
# ------------------ COLAB SETUP ------------------
from google.colab import files
import zipfile
import os

# Step 1: Upload your ZIP file
uploaded = files.upload()  # choose your zip file

zip_path = list(uploaded.keys())[0]
print("Uploaded:", zip_path)

# Step 2: Unzip
unzip_dir = "/content"
os.makedirs(unzip_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

# List files in unzipped folder
import glob
print("Files in unzipped folder:", glob.glob(unzip_dir + "/*"))

# Assume the CSV is named sqli_updated.csv inside the ZIP
csv_path = glob.glob(unzip_dir + "/*.csv")[0]
print("Using CSV file:", csv_path)



Saving rbsqli.zip to rbsqli.zip
Uploaded: rbsqli.zip
Files in unzipped folder: ['/content/sql_injectiondataset_final_updated.csv', '/content/rbsqli.zip', '/content/sample_data']
Using CSV file: /content/sql_injectiondataset_final_updated.csv


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    confusion_matrix,
    classification_report
)
import joblib
import os

# ---------------- CONFIG ----------------
dataset_path = "/content/sql_injectiondataset_final_updated.csv"
random_seed = 42
output_dir = "/sqli_models_multiclass"
os.makedirs(output_dir, exist_ok=True)

# TF-IDF / model hyperparameters
tfidf_max_features = 50000
tfidf_ngram_range = (1, 2)
knn_k = 5
svc_kernel = 'linear'
svc_C = 1.0
nb_alpha = 1.0
rf_n_estimators = 200
rf_max_depth = None
lr_max_iter = 1000

# ---------------- LOAD DATASET ----------------
df = pd.read_csv(dataset_path, low_memory=False)
df = df[['sql_query', 'injection_type', 'vulnerability_status']]

# ---------------- BASE TEST COUNTS ----------------
test_counts = {
    "None_Type": 7352,
    "Error-Based": 808,
    "Time-Based": 554,
    "meta_based": 472,
    "Union-Based": 390,
    "stackqueries_based": 220,
    "boolean-based": 204
}

# ---------------- SCALE COUNTS FOR TRAIN/VALID ----------------
train_counts = {k: v * 7 for k, v in test_counts.items()}
valid_counts = {k: v * 2 for k, v in test_counts.items()}

dataset_sizes = {
    "train": train_counts,
    "valid": valid_counts,
    "test": test_counts
}

# ---------------- FUNCTION TO SAMPLE DATA ----------------
def make_split(counts_dict):
    sampled_df = pd.DataFrame()
    for inj_type, n_rows in counts_dict.items():
        temp = df[df['injection_type'] == inj_type].sample(
            n=n_rows, random_state=random_seed, replace=True
        )
        sampled_df = pd.concat([sampled_df, temp])
    final_df = sampled_df.sample(frac=1.0, random_state=random_seed).reset_index(drop=True)
    final_df['label'] = final_df['injection_type']
    return final_df

# ---------------- CREATE SPLITS ----------------
train_df = make_split(dataset_sizes["train"])
valid_df = make_split(dataset_sizes["valid"])
test_df  = make_split(dataset_sizes["test"])

print(f"Train size: {len(train_df)}, Valid size: {len(valid_df)}, Test size: {len(test_df)}")
print("Train label distribution:\n", train_df['label'].value_counts())
print("Valid label distribution:\n", valid_df['label'].value_counts())
print("Test label distribution:\n", test_df['label'].value_counts())

# ---------------- TF-IDF VECTORIZE ----------------
vectorizer = TfidfVectorizer(
    max_features=tfidf_max_features,
    ngram_range=tfidf_ngram_range,
    lowercase=True,
    token_pattern=r"(?u)\b\w+\b"
)

X_train = vectorizer.fit_transform(train_df['sql_query'].astype(str))
X_valid = vectorizer.transform(valid_df['sql_query'].astype(str))
X_test  = vectorizer.transform(test_df['sql_query'].astype(str))

y_train = train_df['label'].values
y_valid = valid_df['label'].values
y_test  = test_df['label'].values

print("\nVectorization complete. Train shape:", X_train.shape)

# ---------------- TRAIN MODELS ----------------
models = {
    "MultinomialNB": MultinomialNB(alpha=nb_alpha),
    "KNN": KNeighborsClassifier(n_neighbors=knn_k, n_jobs=-1),
    "RandomForest": RandomForestClassifier(
        n_estimators=rf_n_estimators, max_depth=rf_max_depth,
        random_state=random_seed, n_jobs=-1
    ),
    "LogisticRegression": LogisticRegression(
        max_iter=lr_max_iter, solver='saga', random_state=random_seed, n_jobs=-1
    ),
    "SVC": SVC(kernel=svc_kernel, C=svc_C, probability=False, random_state=random_seed)
}

# ---------------- TRAIN + EVALUATE ----------------
results = {}
for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train)

    y_pred_test = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred_test)
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_test, y_pred_test, average='macro', zero_division=0
    )
    cm = confusion_matrix(y_test, y_pred_test, labels=model.classes_)

    print("\n" + "-"*50)
    print(f"Model: {name}")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision (macro): {prec:.4f}")
    print(f"Recall (macro): {rec:.4f}")
    print(f"F1 (macro): {f1:.4f}")
    print("Confusion Matrix:\n", cm)
    print("\nClassification Report:\n", classification_report(
        y_test, y_pred_test, zero_division=0, target_names=model.classes_
    ))

    results[name] = model

# ---------------- SAVE MODELS ----------------
joblib.dump(vectorizer, os.path.join(output_dir, "tfidf_vectorizer.joblib"))
for name, model in results.items():
    joblib.dump(model, os.path.join(output_dir, f"{name}_model.joblib"))

print(f"\nSaved vectorizer and models to {output_dir}")


Train size: 70000, Valid size: 20000, Test size: 10000
Train label distribution:
 label
None_Type             51464
Error-Based            5656
Time-Based             3878
meta_based             3304
Union-Based            2730
stackqueries_based     1540
boolean-based          1428
Name: count, dtype: int64
Valid label distribution:
 label
None_Type             14704
Error-Based            1616
Time-Based             1108
meta_based              944
Union-Based             780
stackqueries_based      440
boolean-based           408
Name: count, dtype: int64
Test label distribution:
 label
None_Type             7352
Error-Based            808
Time-Based             554
meta_based             472
Union-Based            390
stackqueries_based     220
boolean-based          204
Name: count, dtype: int64

Vectorization complete. Train shape: (70000, 1151)

Training MultinomialNB...

--------------------------------------------------
Model: MultinomialNB
Accuracy: 0.9586
Precision (macro): 